In [42]:
import os
from dotenv import load_dotenv, find_dotenv
import tweepy as tw
import matplotlib.pyplot as plt
import pandas as pd
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer

#### Load OAuth 2.0 Bearer Token

In [43]:
load_dotenv(find_dotenv())
bearer_token = os.getenv('BEARER_TOKEN')

In [44]:
client = tw.Client(bearer_token,  return_type = dict, wait_on_rate_limit=True)

In [45]:
def get_tweets(max_tweets=10):
    # Define the search term
    query = '(Bitcoin OR #Bitcoin OR #BTC OR #btc) lang:en -is:retweet'   
    tweets = client.search_recent_tweets(query=query, tweet_fields=['text'], max_results=max_tweets)
    #print(tweets)
    return [tweet['text'] for tweet in tweets['data']]


In [46]:
posts = get_tweets()
# print(posts)

### Load the model

https://huggingface.co/ElKulako/cryptobert#:~:text=CryptoBERT%20is%20a%20pre%2Dtrained,cryptocurrency%2Drelated%20social%20media%20posts

In [47]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

### Get the sentiment scores

In [48]:
preds = pipe(posts)

In [49]:
df = pd.DataFrame(preds)
df.insert(0, "tweet", posts, True)
df

,tweet,label,score
0,#bitcoin is now $19077 #19K https://t.co/N8U6z...,Neutral,0.929488
1,The NTK token will be used to power Netkoin’s ...,Bullish,0.728720
2,@crypto2real #SpookyShiba will explode over th...,Bullish,0.653898
3,Big news: Companies can value Bitcoin at its c...,Bullish,0.502303
4,@AltCryptoGems #SpookyShiba will explode over ...,Bullish,0.751607
5,"@Excellion All systems go Commander Mow, steal...",Bullish,0.656621
6,Join me at Bybit and win Mystery Boxes togethe...,Neutral,0.737681
7,A complete US ban on Russian aluminum threaten...,Neutral,0.712163
8,Bitcoin 1-week activity: \nPrice: $19.08K -6.2...,Neutral,0.947605
9,"Unique, modern, win-win trading strategies,\nd...",Neutral,0.796104
